# Fine-tune LLaMA 2 models on SageMaker JumpStart #1: Deploying & Invoking LLaMa-2

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook.

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-west-2/introduction_to_amazon_algorithms|jumpstart-foundation-models|llama-2-text-completion.ipynb)

---
In this demo notebook, we demonstrate how to use the SageMaker Python SDK to deploy pre-trained Llama 2 model as well as fine-tune it for your dataset in domain adaptation or instruction tuning format.

---

### Model License information
---
To perform inference on these models, you need to pass custom_attributes='accept_eula=true' as part of header. This means you have read and accept the end-user-license-agreement (EULA) of the model. EULA can be found in model card description or from https://ai.meta.com/resources/models-and-libraries/llama-downloads/. By default, this notebook sets custom_attributes='accept_eula=false', so all inference requests will fail until you explicitly change this custom attribute.

Note: Custom_attributes used to pass EULA are key/value pairs. The key and value are separated by '=' and pairs are separated by ';'. If the user passes the same key more than once, the last value is kept and passed to the script handler (i.e., in this case, used for conditional logic). For example, if 'accept_eula=false; accept_eula=true' is passed to the server, then 'accept_eula=true' is kept and passed to the script handler.

---

### Set up

---
We begin by installing and upgrading necessary packages. Restart the kernel after executing the cell below for the first time.

---

In [2]:
!pip install --upgrade sagemaker datasets

  Using cached datasets-2.16.1-py3-none-any.whl.metadata (20 kB)
  Using cached urllib3-1.26.18-py2.py3-none-any.whl.metadata (48 kB)
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl.metadata (3.6 kB)
  Using cached xxhash-3.4.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached fsspec-2023.10.0-py3-none-any.whl.metadata (6.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 9.0 MB/s eta 0:00:00:00:0100:01
Using cached datasets-2.16.1-py3-none-any.whl (507 kB)
Using cached fsspec-2023.10.0-py3-none-any.whl (166 kB)
Using cached urllib3-1.26.18-py2.py3-none-any.whl (143 kB)
Using cached pyarrow_hotfix-0.6-py3-none-any.whl (7.9 kB)
Using cached xxhash-3.4.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (194 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7
  Attempting uninstall: fsspec
    Found existing installatio

## Deploy Pre-trained Model

---

First we will deploy the Llama-2 model as a SageMaker endpoint. To train/deploy 13B and 70B models, please change model_id to "meta-textgeneration-llama-2-7b" and "meta-textgeneration-llama-2-70b" respectively.

---

In [3]:
model_id, model_version = "meta-textgeneration-llama-2-7b", "*"

If you are deploying the model for the first time, make sure to follow the code below to deploy the model endpoint and then use it to make predictions.

In [4]:
from sagemaker.jumpstart.model import JumpStartModel

pretrained_model = JumpStartModel(model_id=model_id)
pretrained_predictor = pretrained_model.deploy(accept_eula = True)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


Using model 'meta-textgeneration-llama-2-7b' with wildcard version identifier '*'. You can pin to version '3.0.2' for more stable results. Note that models may have different input/output signatures after a major version upgrade.


---------!

If you have already deployed the model and do not wish to deploy again, uncomment the code below to utilize the existing endpoint to make predictions based on the user query.

In [5]:
# from sagemaker.predictor import Predictor
# from sagemaker.serializers import JSONSerializer
# from sagemaker.deserializers import JSONDeserializer

# # Use the existing endpoint name
# endpoint_name = "<endpoint name>"  # Replace with your endpoint name

# # Create a SageMaker predictor object
# pretrained_predictor = Predictor(
#     endpoint_name=endpoint_name,
#     serializer=JSONSerializer(),
#     deserializer=JSONDeserializer(),
# )

# name = pretrained_predictor.endpoint 

## Invoke the endpoint

---
Next, we invoke the endpoint with some sample queries. Later, in this notebook, we will fine-tune this model with a custom dataset and carry out inference using the fine-tuned model. We will also show comparison between results obtained via the pre-trained and the fine-tuned models.

---

In [6]:
def print_response(payload, response):
    print(payload["inputs"])
    print(f"> {response[0]['generated_text']}")
    print("\n==================================\n")

In [7]:
payload = {
    "inputs": "I believe the meaning of life is",
    "parameters": {
        "max_new_tokens": 200,
        "top_p": 0.9,
        "temperature": 0.9,
        "return_full_text": True,
    },
}
try:
    response = pretrained_predictor.predict(payload, custom_attributes="accept_eula=true")
    print_response(payload, response)
except Exception as e:
    print(e)

I believe the meaning of life is
> I believe the meaning of life is to achieve complete joy, to be happy and to help others become happy, not only during one’s lifetime, but after it as well.
This is the task of every saint, and that is the nature of my work. I am a spiritual teacher, and through the practice of my path I have learned to achieve this goal.
I have learned to deal with all negative emotions, such as anger, fear, and insecurity, and how to overcome them. I have achieved great inner strength, which allows me to be fully present in this world. I am at peace and do not identify with any of the problems and suffering that life offers.
I have learned how to focus my attention inward and develop a direct connection with God. I have realized that my own life has a higher meaning, and that I am living it according to God’s plan. I have achieved the purpose of my life.
This is what the meaning of life is to me. I believe




## Clean-up
If you are not going to run the fine-tuning Llama2 lab (Finetuning_LLaMa-2.ipynb) then delete the endpoint by uncommenting and running the cell below. Otherwise carry on to the fine-tuning Llama2 lab, where this endpoint will also be used.

In [8]:
# predictor.delete_model()
# predictor.delete_endpoint()

---
To learn about additional use cases of pre-trained model, please checkout the notebook [Text completion: Run Llama 2 models in SageMaker JumpStart](https://github.com/aws/amazon-sagemaker-examples/blob/main/introduction_to_amazon_algorithms/jumpstart-foundation-models/llama-2-text-completion.ipynb).

---